In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import sklearn

In [2]:
df_featuresonly = pd.read_excel("wc_data_ready_featuresonly.xlsx", engine="openpyxl")
df_featurestarget = pd.read_excel("wc_data_ready_features_target.xlsx", engine="openpyxl")

(target classes)  
0 is cloudy  
1 is rainy  
2 is snowy  
3 is sunny

As decided on 10.1, to make the prediction more meaningful I'll now remove the cloud cover and visibility features, and continue that way.  

In [3]:
df_featuresonly.drop(
    ["Visibility (km)", "Cloud Cover_clear", "Cloud Cover_cloudy", "Cloud Cover_overcast", 
     "Cloud Cover_partly cloudy"], axis=1, inplace=True)
df_featurestarget.drop(
    ["Visibility (km)", "Cloud Cover_clear", "Cloud Cover_cloudy", "Cloud Cover_overcast", 
     "Cloud Cover_partly cloudy"], axis=1, inplace=True)

In [4]:
df_featurestarget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12318 entries, 0 to 12317
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Temperature (Celsius)       12318 non-null  float64
 1   Precipitation (%)           12318 non-null  float64
 2   Wind Speed (km/h)           12318 non-null  float64
 3   Atmospheric Pressure (hPa)  12318 non-null  float64
 4   Humidity (%)                12318 non-null  float64
 5   Weather Type                12318 non-null  int64  
 6   Season_Autumn               12318 non-null  bool   
 7   Season_Spring               12318 non-null  bool   
 8   Season_Summer               12318 non-null  bool   
 9   Season_Winter               12318 non-null  bool   
 10  Location_coastal            12318 non-null  bool   
 11  Location_inland             12318 non-null  bool   
 12  Location_mountain           12318 non-null  bool   
dtypes: bool(7), float64(5), int64(1

In [5]:
from sklearn.model_selection import train_test_split
X = df_featuresonly
y = df_featurestarget["Weather Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=40)



In [6]:
# Multi_class parameter is deprecated. I think the function automatically selects multinomial LogReg when 
# number of target classes >= 3 (documentation) 

# Shuffling the data not necessary with lbfgs solver (documentation, so should be no need for random_state parameter

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')

In [7]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

y_train_pred = cross_val_predict(model, X_train, y_train, cv=5)
print(classification_report(y_train, y_train_pred, output_dict=True))

# Note: the documentation says, if you specify dict output, you cannot round the values with the rounding parameter 

{'0': {'precision': 0.7679195804195804, 'recall': 0.7203772037720377, 'f1-score': 0.7433890416754813, 'support': 2439.0}, '1': {'precision': 0.8039545645771982, 'recall': 0.7825552825552825, 'f1-score': 0.793110603859722, 'support': 2442.0}, '2': {'precision': 0.8642384105960265, 'recall': 0.921537857983523, 'f1-score': 0.8919688627302069, 'support': 2549.0}, '3': {'precision': 0.8041278834479968, 'recall': 0.8197194719471947, 'f1-score': 0.8118488253319714, 'support': 2424.0}, 'accuracy': 0.8122589811244165, 'macro avg': {'precision': 0.8100601097602005, 'recall': 0.8110474540645095, 'f1-score': 0.8100793333993453, 'support': 9854.0}, 'weighted avg': {'precision': 0.8106720723998467, 'recall': 0.8122589811244165, 'f1-score': 0.8109856049295653, 'support': 9854.0}}


In [10]:
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred, output_dict=True))

{'0': {'precision': 0.7589134125636672, 'recall': 0.7039370078740157, 'f1-score': 0.7303921568627451, 'support': 635.0}, '1': {'precision': 0.8085443037974683, 'recall': 0.797191887675507, 'f1-score': 0.802827965435978, 'support': 641.0}, '2': {'precision': 0.8475894245723172, 'recall': 0.9159663865546218, 'f1-score': 0.8804523424878837, 'support': 595.0}, '3': {'precision': 0.7933333333333333, 'recall': 0.8026981450252951, 'f1-score': 0.7979882648784576, 'support': 593.0}, 'accuracy': 0.8031655844155844, 'macro avg': {'precision': 0.8020951185666966, 'recall': 0.8049483567823599, 'f1-score': 0.8029151824162661, 'support': 2464.0}, 'weighted avg': {'precision': 0.8015216274347814, 'recall': 0.8031655844155844, 'f1-score': 0.8017402314551629, 'support': 2464.0}}
